In [1]:
import keras

Using TensorFlow backend.


In [2]:
#from keras.datasets import mnist

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import Adam

In [4]:
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

In [5]:
import pickle
import numpy as np
import pandas as pd
#from sklearn.model_selection import train_test_split # ホールドアウト法に関する関数
#import matplotlib.pyplot as plt
#from collections import OrderedDict
#from common.layers import Convolution, MaxPooling, ReLU, Affine, Dropout, SoftmaxWithLoss
#from common.optimizer import RMSProp, SGD, Adam
#from common.nn import SimpleConvNet
#from common.deep_convnet import DeepConvNet

In [6]:
from keras.models import load_model

In [7]:
train_data = np.load("./train_data_mr.npy")
train_label = np.load("./train_label_mr.npy")
print(train_data.shape)
print(train_label.shape)

(59951, 1, 28, 28)
(59951, 15)


In [8]:
test_size = 0.2        # 全データのうち、何%をテストデータにするか（今回は20%に設定）
X_train, X_test, y_train, y_test = train_test_split(train_data, train_label, test_size=test_size, random_state=1234) # ホールドアウト法を実行（テストデータはランダム選択）


In [9]:
# 正規化
X_train = (X_train - X_train.min()) / (X_train.max() - X_train.min())
X_train = X_train.astype('float64')


In [10]:
x = X_train
t = y_train

#x = x.reshape(-1,1,28,28) # 配列形式の変換
x = x.transpose(0, 2, 3, 1)
X_test = X_test.transpose(0, 2, 3, 1)

#epochs = 200
#batch_size = 100

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train1, x_valid, y_train1, y_valid = train_test_split(x_train, y_train, test_size=0.175)
x_train = x_train1
y_train = y_train1

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32')/255
x_valid = x_valid.reshape(x_valid.shape[0], 28, 28, 1).astype('float32')/255
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float32')/255

# convert one-hot vector
y_train = keras.utils.to_categorical(y_train, 10)
y_valid = keras.utils.to_categorical(y_valid, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [11]:
# create model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(15, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

print(model.summary())






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
____________________

In [ ]:
# callback function
#plot_losses = PlotLosses()      # グラフ表示(live plot)
#plot_losses.epochs = epochs
#csv_logger = CSVLogger('trainlog.csv')

# train
history = model.fit(x, t,
                    batch_size=100, epochs=56,
                    verbose=1,
                    validation_data=(X_test, y_test))

# result
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss: {0}'.format(score[0]))
print('Test accuracy: {0}'.format(score[1]))

plot_result(history)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 47960 samples, validate on 11991 samples
Epoch 1/56
47960/47960 [==============================] - 37s 764us/step - loss: 0.6597 - acc: 0.7981 - val_loss: 0.7660 - val_acc: 0.9515
Epoch 2/56
47960/47960 [==============================] - 37s 764us/step - loss: 0.2290 - acc: 0.9275 - val_loss: 0.6092 - val_acc: 0.9617
Epoch 3/56
47960/47960 [==============================] - 37s 771us/step - loss: 0.1612 - acc: 0.9490 - val_loss: 0.4510 - val_acc: 0.9716
Epoch 4/56
47960/47960 [==============================] - 37s 772us/step - loss: 0.1269 - acc: 0.9561 - val_loss: 0.4741 - val_acc: 0.9701
Epoch 5/56
47960/47960 [==============================] - 37s 780us/step - loss: 0.1084 - acc: 0.9632 - val_loss: 0.4472 - val_acc: 0.9718
Epoch 6/56
47960/47960 [==============================] - 38s 782us/step - loss: 0.0904 - acc: 0.9698 - val_loss: 0.4021 - val_acc: 0.9749
Epoch 7/56
47960/47960

In [ ]:
model.save('model.h5')